In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from gensim.models.doc2vec import TaggedDocument
import gensim.models.doc2vec
import pickle
from gensim.models import Doc2Vec
import gensim.corpora as corpora
import spacy

C:\Users\Hesham\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv('./data_clean.csv')
with open('./docs.pkl', 'rb') as file:
    docs = pickle.load(file)

In [3]:
word_list = [doc.words for doc in docs]

In [4]:
bigram = gensim.models.Phrases(word_list)

In [5]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [6]:
with open('./bi_phraser.pkl', 'wb') as file:
    pickle.dump(bigram_mod, file)

In [7]:
# with open('./bi_phraser.pkl', 'rb') as file:
#     bigram_mod = pickle.load(file)

In [8]:
bi_docs = []
for doc in docs:
    words = bigram_mod[doc.words]
    tags = bigram_mod[doc.tags]
    bi_docs.append(TaggedDocument(words, tags))
word_list = [doc.words for doc in bi_docs]

In [9]:
with open('./docs_bigram.pkl', 'wb') as file:
    pickle.dump(bi_docs, file)

In [10]:
trigram = gensim.models.Phrases(word_list)

In [11]:
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [12]:
with open('./tri_phraser.pkl', 'wb') as file:
    pickle.dump(trigram_mod, file)

In [13]:
# with open('./tri_phraser.pkl', 'rb') as file:
#     trigram_mod = pickle.load(file)

In [14]:
tri_docs = []
for doc in bi_docs:
    words = trigram_mod[doc.words]
    tags = trigram_mod[doc.tags]
    tri_docs.append(TaggedDocument(words, tags))

In [15]:
with open('./docs_trigram.pkl', 'wb') as file:
    pickle.dump(tri_docs, file)

In [16]:
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])
# nlp.max_length = 1600000
# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [17]:
# with open('./docs_bigram_lemmatized.pkl', 'wb') as file:
#     pickle.dump(data_lemmatized, file)

In [39]:
# tag = ''
# if tag in doc.tags:
#     f = dict()
#     for word in doc.words:
#         if not word in f.keys():
#             f[word] = 1
#         else:
#             f[word] = f[word]+1
#     display(sorted(f.items(), key=lambda x:x[1], reverse=True))

In [36]:
proc_docs = []
blacklist = ['product', 'home_page', 'shop', 'order', 'contact', 'sale', 'price', 'navig_menu', 'add_cart', 'onlin', 'search',
             'buyer', 'seller', 'save', 'account_sign']
for doc in tri_docs:
    f = dict()
    words = [w for w in doc.words if not w in blacklist]
    for word in words:
        if not word in f.keys():
            f[word] = 1
        else:
            f[word] = f[word]+1
    f = sorted(f.items(), key=lambda x:x[1], reverse=True)
    tags = [doc.tags[0]]
    tags += [f[0][0], f[1][0], f[2][0]]
    proc_docs.append(TaggedDocument(words, tags))

In [38]:
with open('./docs_proc.pkl', 'wb') as file:
    pickle.dump(proc_docs, file)

In [37]:
proc_docs[0]

TaggedDocument(words=['retro', 'planet', 'tin_sign', 'retro', 'decor', 'diner_furnitur', 'vintag', 'sign', 'uniqu_gift_idea', 'free_ship_tax', 'free', 'login', 'wish_list', 'view_cart_checkout', 'blog', 'cool', 'retro', 'gift', 'decor', 'item', 'wall_decor', 'metal', 'sign', 'wall_decal', 'floor', 'decal', 'clock_thermomet', 'sign', 'bundl', 'tryptych', 'neon_sign', 'poster_print', 'wood', 'sign', 'canva_wall_art', 'film_cell', 'gold', 'record', 'sign', 'decal', 'set', 'gift', 'sticker', 'mileston_birthday', 'gift_set', 'gift_basket', 'gift_wrap', 'tape', 'greet_card', 'tag', 'joke', 'gag_gift', 'toy', 'hero', 'geek', 'techi', 'pet_pet_lover', 'keychain', 'button_pin', 'kitchen_dine', 'bar', 'coffe_mug', 'tiki_mug', 'deli', 'serv', 'basket', 'kitchen_linen', 'kitchen_canist', 'jar', 'salt_pepper_shaker', 'glasswar', 'dispens', 'restaurantwar', 'tablewar_kitchen', 'drinkwar', 'bar', 'loung', 'parti', 'home_decor', 'funki', 'stuff', 'doormat', 'refriger_magnet', 'woven_throw', 'blanket',

In [ ]:
removed = []
urls = data.url.unique()
for idx, row in pd.concat(docs).iterrows():
    if row.url not in urls:
        removed.append(row.url)
df = pd.DataFrame(removed)
df.to_csv('removed.csv')

print(len(removed))
with open('removed.json', 'w') as file:
    file.write('{"removed_sites" : [')
    file.write(','.join(['"'+v.values[0]+'"' for idx, v in df.iterrows()]))
    file.write(']}')